In [1]:
import findspark

In [2]:
findspark.init('/home/ubuntu/spark-2.4.4-bin-hadoop2.7/')

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('LReg').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('data/Ecommerce-Customers.csv', inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [8]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [9]:
data.count()

500

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [12]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
                                     'Time on App',
                                     'Time on Website',
                                     'Length of Membership'], 
                           outputCol="features")

In [13]:
output = assembler.transform(data)

In [14]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [15]:
final_data = output.select("features", "Yearly Amount Spent")

In [16]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

## Split to train, test data

In [17]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [18]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                345|
|   mean| 498.48606204859175|
| stddev|  77.06919843880166|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [19]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                155|
|   mean|  501.1569530485886|
| stddev|  84.32774662585733|
|    min|  275.9184206503857|
|    max|  725.5848140556806|
+-------+-------------------+



In [20]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [21]:
lr_model = lr.fit(train_data)

In [22]:
test_result = lr_model.evaluate(test_data)

In [23]:
test_result.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 10.875539119153132|
|  5.408825068080091|
|-12.610319145406208|
| 10.422291221008663|
|  22.52484969388547|
| 19.906561465185007|
| -3.853905756020879|
|  4.699351614424188|
|-1.8082685347437177|
| 2.3511214474704616|
|-10.937473894596906|
| -2.416394603252627|
|  7.800296746580159|
|  8.270266468626971|
|-16.743178116885304|
|  -17.1838609878439|
| 0.5992401361272073|
|  4.345623934741923|
| 17.765303148268686|
| 1.0817727415414993|
+-------------------+
only showing top 20 rows



In [24]:
test_result.rootMeanSquaredError

9.19263269490612

In [25]:
test_result.r2

0.9880394871591076

In [26]:
unlabeled_data = test_data.select('features')

In [27]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.7377203726281...|
|[31.0472221394875...|
|[31.0662181616375...|
|[31.1695067987115...|
|[31.2834474760581...|
|[31.3123495994443...|
|[31.5171218025062...|
|[31.5316044825729...|
|[31.7216523605090...|
|[31.7366356860502...|
|[31.8093003166791...|
|[31.8186165667690...|
|[31.8209982016720...|
|[31.8512531286083...|
|[31.9048571310136...|
|[31.9563005605233...|
|[32.0047530203648...|
|[32.0123007682454...|
|[32.0180740106320...|
|[32.0444861274404...|
+--------------------+
only showing top 20 rows



In [28]:
predictions = lr_model.transform(unlabeled_data)

In [29]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.7377203726281...|450.90520307707675|
|[31.0472221394875...| 387.0885741209413|
|[31.0662181616375...|461.54361235308056|
|[31.1695067987115...|416.93423958128415|
|[31.2834474760581...|  569.256239731782|
|[31.3123495994443...| 443.6848565627556|
|[31.5171218025062...| 279.7723264064066|
|[31.5316044825729...| 431.8162541149384|
|[31.7216523605090...|349.58519516661636|
|[31.7366356860502...|494.58232480806146|
|[31.8093003166791...| 547.7093732574381|
|[31.8186165667690...| 448.8350679733883|
|[31.8209982016720...| 416.8749842666332|
|[31.8512531286083...| 464.7219801981714|
|[31.9048571310136...|490.69303553970144|
|[31.9563005605233...| 564.3097927350427|
|[32.0047530203648...|463.14674098450223|
|[32.0123007682454...| 488.5994291312163|
|[32.0180740106320...| 340.0178075970466|
|[32.0444861274404...| 447.1480564450085|
+--------------------+------------